In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import torch.nn as nn
import torch
import pytorch_lightning as pl
from bayesian_nnet.net_utils import initialize_pl_trainer
from bayesian_nnet.data import IQDataModel
from bayesian_nnet.network import AutoModClassifier
from bayesian_nnet.data import initialize_data_splits

### Initialize data split indexing

In [11]:
out_of_distrib_modes = ["lsb",
                        "usb"]

initialize_data_splits(out_of_distrib_modes)

In [17]:
iqdata = IQDataModel()
model = AutoModClassifier()
callbacks, logger = initialize_pl_trainer("frequentist_net")

trainer = pl.Trainer(callbacks=callbacks,
                     max_epochs=10,
                     logger=mlf_logger)

trainer.fit(model, iqdata)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


FileNotFoundError: [Errno 2] No such file or directory: '/home/mspcvsp/Data/panoradio_hf/train_data_tags.csv'